In [1]:
# -*- coding: utf-8 -*-
#  Copyright 2024 -  United Kingdom Research and Innovation
#  Copyright 2024 -  The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#   Authored by:    Hannah Robarts (STFC - UKRI)
#                   Laura Murgatroyd (STFC - UKRI)

# Paganin Processor 
This deep-dive contains some examples of using the `PaganinProcessor` phase retrieval methods in CIL.

#### Phase contrast imaging
Phase contrast imaging is commonly used in tomography as a tool to exploit the different contrast provided by absorption and phase.

In absorption contrast imaging, variation in beam intensity $I$ is given by,
$$
I  = I_0\exp(-\mu T),
$$
where $T$ is the material thickness and $\mu$ is the material linear attenuation coefficient. We can also express $\mu$ in terms of the **complex** part of the material refractive index $\beta$ and the wavevector of the x-rays (or other probe)  $k$,

$$
\mu = 2k\beta
$$

Variation in material phase $\phi$ is given by,
$$
\phi = -k\delta T,
$$
where $\delta$ is the **real** part of the material refractive index.

Phase contrast imaging is commonly used in samples which give poor absorption contrast, like light element materials, to resolve details which might not otherwise be resolved with absorption contrast alone.

#### Propagation-based phase contrast
There are a number of hardware-based methods to measure phase but the simplest methods to enable phase contrast imaging are propagation-based methods. These rely on the fact that a phase shift caused by the sample results in the x-rays (or other probe) propagating differently, or refracting, which is measured as a variation in intensity at the detector after some propagation distance $\Delta$. This causes bright fringes around edges in the sample, known as edge enhancement. There are a number of specialist lab-based CT systems that are designed to exploit propagation based phase contrast and it's also common to see phase effects in synchrotron systems with long propagation distances. These can be really helpful for highlighting features in a sample, for example for segmentation.

#### Phase retrieval
However, edge-enhancement changes the intensity profile in the projections by combining the absorption and phase information. So in certain circumstances - for example where quantitative measurement of sample features is required - phase retrieval methods are used to separate these effects. This notebook uses the commonly used Paganin phase retrieval method which accounts for the change in intensity due to phase effects to retrieve the sample thickness from phase contrast images [[1](https://onlinelibrary.wiley.com/doi/10.1046/j.1365-2818.2002.01010.x)]. The form of the phase retrieval additionally acts as a filter on the data, which if used with the correct physical parameters, results in a boost to the signal to noise ratio (SNR) without losing spatial resolution. Paganin phase retrieval is therefore also commonly used as a filter to boost SNR in many different contexts.

[1] D. Paganin et al. "Simultaneous phase and amplitude extraction from a single defocused image of a homogeneous object." Journal of Microscopy, 206 (2002): 33-40.  [DOI:10.1046/j.1365-2818.2002.01010.x](https://doi.org/10.1046/j.1365-2818.2002.01010.x)

This notebook requires CIL v24.1.0 or greater, check the version below

In [ ]:
import cil
print(cil.__version__)

Load some dependencies

In [ ]:
from cil.processors import PaganinProcessor, TransmissionAbsorptionConverter
from cil.utilities.display import show2D, show_geometry
from cil.recon import FBP
import numpy as np
import matplotlib.pyplot as plt 
from cil.io import NEXUSDataReader

#### TomoBank example

This example demonstrates phase retrieval on a real dataset: tomo_00068 from the TomoBank [[2](http://www.doi.org/10.1088/1361-6501/aa9c19)] wet sample dataset [[3](https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#wet-sample)]. The data were collected at the SYRMEP beamline of the Elettra synchotron on a bone tissue composite sample, a description of the experiment is given in [[4](https://link.springer.com/chapter/10.1007/978-3-319-19387-8_70)].

A modified version of the TomoBank dataset can be retrieved using the command below. We have binned and cropped the data to make it more manageable, then normalised the data and applied a centre of rotation correction:

`wget https://tomography.stfc.ac.uk/notebooks/phase/tomo_000068_binned.nxs`

[2] F. De Carlo et al. “TomoBank: a tomographic data repository for computational x-ray science.” Measurement Science and Technology 29.3 (2018): 034004. [DOI:10.1088/1361-6501/aa9c19](http://www.doi.org/10.1088/1361-6501/aa9c19)

[3] https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#wet-sample

[4] F. Brun et al. "A synchrotron radiation microtomography study of wettability and swelling of nanocomposite Alginate/Hydroxyapatite scaffolds for bone tissue engineering"  World Congress on Medical Physics and Biomedical Engineering, June 7-12, 2015, Toronto, Canada (pp.288-291) [DOI:10.1007/978-3-319-19387-8_70 ](http://dx.doi.org/10.1007/978-3-319-19387-8_70)

Load the data using `NEXUSDataReader`, (if you're running this locally you will need to change the filename to the path where you downloaded it)

In [ ]:
filename = '/mnt/materials/SIRF/Fully3D/CIL/Phase/tomo_000068_binned.nxs' 
data = NEXUSDataReader(filename).read()

Take a look at the dataset using `show2D`

In [ ]:
show2D(data)

Print the dataset geometry parameters and plot the source, sample and detector positions using `show_geometry()`.

In [ ]:
print(data.geometry)
show_geometry(data.geometry)

The propagation distance is an important parameter in propagation-based phase contrast imaging as it determines how far the beam is refracted and therefore the extent of the edge enhancement. In this dataset we can see there is a long propagation between the sample (red) and detector (blue).

Get a slice of the data, convert to absorption and reconstruct the dataset using filtered back projection. Then use `show2D()` to view the reconstruction and zoom in on some sample features, in this case air bubbles in the bone composite material.

In [ ]:
data_slice = data.get_slice(vertical='centre')
data_slice = TransmissionAbsorptionConverter()(data_slice)
fbp =  FBP(data_slice)
recon = fbp.run(verbose=0)
show2D([recon, recon.array[200:300, 350:450]],
       axis_labels=recon.dimension_labels)

There are some edge enhancements in this dataset, take a closer look by plotting a cross-section through the reconstruction

In [ ]:
plt.plot(recon.array[200:300,400])
plt.xlabel('horizontal_x')
plt.ylabel('Intensity')

Next we run the phase retrieval on the same data with the `PaganinProcessor` which is implemented based on [[1](https://onlinelibrary.wiley.com/doi/10.1046/j.1365-2818.2002.01010.x)]. The processor returns the material retrieved thickness $T$, removing the effect of phase in the image

$$
T(x,y) = - \frac{1}{\mu}\ln\left (\mathcal{F}^{-1}\left 
        (\frac{\mathcal{F}\left ( M^2I_{norm}(x, y,z = \Delta) \right )}{1 + 
          \alpha\left ( k_x^2 + k_y^2 \right )}  \right )\right )
$$

where
- $\mu = \frac{4\pi\beta}{\lambda}$ is the material linear 
attenuation coefficient where $\beta$ is the complex part of the 
material refractive index and $\lambda=\frac{hc}{E}$ is the probe 
wavelength,
- $M$ is the magnification at the detector,
- $I_{norm}$ is the input image which is expected to be the 
normalised transmission data, 
- $\Delta$ is the propagation distance,
- $\alpha = \frac{\Delta\delta}{\mu}$ is a parameter determining 
the strength of the filter to be applied in Fourier space where 
$\delta$ is the real part of the deviation of the material 
refractive index from 1 
- $k_x, k_y = \left ( \frac{2\pi p}{N_xW}, \frac{2\pi q}{N_yW} 
\right )$ where $p$ and $q$ are co-ordinates in a Fourier 
mesh in the range $-N_x/2$ to $N_x/2$ and $-N_y/2$ to $N_y/2$ for an image with 
size $N_x, N_y$ and pixel size $W$.
- $\mathcal{F}$ represents the Fourier transform and $\mathcal{F}^{-1}$ is the inverse Fourier transform

We need to set up the `PaganinProcessor` with the physical parameters for this experiment. 
- We can find the experiment energy in [[2](https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#multi-distance)].
- We can get refractive indices for real materials at x-ray wavelengths at [[5](https://henke.lbl.gov/optical_constants/getdb2.html)]. We don't know the refractive indices for this sample so we start with a guess and tune this parameter later.
- We use the distance parameters that are stored in `data.geometry` these are propagation distance, pixel size and magnification.


[1] D. Paganin et al. "Simultaneous phase and amplitude extraction from a single defocused image of a homogeneous object." Journal of Microscopy, 206 (2002): 33-40.  [DOI:10.1046/j.1365-2818.2002.01010.x](https://doi.org/10.1046/j.1365-2818.2002.01010.x)

[2] F. De Carlo et al. “TomoBank: a tomographic data repository for computational x-ray science.” Measurement Science and Technology 29.3 (2018): 034004. [DOI:10.1088/1361-6501/aa9c19](http://www.doi.org/10.1088/1361-6501/aa9c19)

[5] Lawrence Berkeley National Laboratory, Centre for X-Ray Optics - X-Ray Interactions With Matter: Refractive Indices https://henke.lbl.gov/optical_constants/getdb2.html

In [ ]:
delta = 1
beta = 1e-2
energy = 14
energy_units = 'keV'

processor = PaganinProcessor(delta=delta, beta=beta, energy=energy, energy_units=energy_units)
processor.set_input(data)
try:
    thickness = processor.get_output()
except Exception as e:
    print(e)

We get an error because the distance units are not supplied in the geometry. This is a common problem because distance units are not always needed for other processing or reconstruction steps. We should check if all the important experimental information is stored in the geometry.

In [ ]:
print("Propagation distance = {:.1f} {:s}".format(data.geometry.dist_center_detector, data.geometry.config.units))
print("Pixel size = {:.4} {:s}".format(data.geometry.pixel_size_h, data.geometry.config.units))

The distances are correct (note the pixel size is double compared to [[2](https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#multi-distance)] because we've binned the data) but the units aren't stored, so we should add them

In [ ]:
data.geometry.config.units = 'mm'

Run the processor again, this time also set the processor return units to be the same as the geometry

In [ ]:
processor = PaganinProcessor(delta=delta, beta=beta, energy=energy, energy_units=energy_units, return_units=data.geometry.config.units)
processor.set_input(data)
thickness = processor.get_output()

Get a slice of the phase-retrieved data and reconstruct it, then compare the reconstruction with the one we got from the original data

In [ ]:
data_slice = thickness.get_slice(vertical='centre')
fbp =  FBP(data_slice)
recon_phase = fbp.run(verbose=0)
show2D([recon.array[200:300, 350:450], recon_phase.array[200:300, 350:450]],
       title=['Original reconstruction', 'With phase retrieval'],
       axis_labels=recon.dimension_labels)

We can see the phase retrieval has the effect of blurring the edge of features in the sample which reduces the appearance of the edge. When we compare the cross-section through this reconstruction we should be aware that the Paganin processor returns the material thickness $T$. To get the reconstruction on the same scale as the original data we have to multiply $T$ by the linear attenuation coefficient $\mu$ which we can get from the `PaganinProcessor` by calling `processor.mu`.

In [ ]:
plt.plot(recon.array[200:300,400])
plt.plot(recon_phase.array[200:300,400]*processor.mu)
plt.xlabel('horizontal_x')
plt.ylabel('Intensity')
plt.legend(['Original reconstruction','Phase retrieval'])

We can also approximate the signal to noise of each reconstruction as the mean divided by the standard deviation, and find the phase retreival results in a boost to the SNR

In [ ]:
print("Original reconstruction SNR = {:.3f}".format(np.abs(np.mean(recon.array)/recon.array.std())))
print("Phase retrieved reconstruction SNR = {:.3f}".format(np.abs(np.mean(recon_phase.array*processor.mu)/(recon_phase.array*processor.mu).std())))

We notice that the phase retrieval starts to reduce the size of the fringes. If $\delta$ and $\beta$ are not precisely known, it's common to tune them until the fringes are fully removed but without removing real features in the sample. We can change $\delta$ and $\beta$ directly or vary $\alpha$, a hyper-parameter containing $\delta$, $\beta$ and the propagation distance $\Delta$, $\alpha = \frac{\Delta\delta}{\mu}$. A larger $\alpha$ has a stronger effect on the data

Check the $\alpha$ value we used so far then try varying alpha and checking the effect on the reconstruction

In [ ]:
print('alpha = {:.2}'.format(processor.alpha))
alpha_array = [processor.alpha] # save the alpha value to an array
recon_array = [recon_phase.array[200:300,400]*processor.mu] # save the cross-section of the reconstruction to an array

In [ ]:
# try different values for alpha
alpha = 0.0005
alpha_array.append(alpha)

# run the phase retrieval
thickness = processor.get_output(override_filter={'alpha':alpha})

# run the reconstruction
data_slice = thickness.get_slice(vertical='centre')
fbp =  FBP(data_slice)
recon_phase = fbp.run(verbose=0)

# save the cross-section through the reconstruction 
recon_array.append(recon_phase.array[200:300,400]*processor.mu)

Now plot the cross-sections through the reconstructions. 
You can re-run the above cell with a different $\alpha$ to add another reconstruction to the array for comparison.

In [ ]:
plt.plot(recon.array[200:300,400], label='Original reconstruction')
for i in np.arange(len(alpha_array)):
    plt.plot(recon_array[i], label=r'$\alpha$={:.2}'.format(alpha_array[i]))
plt.xlabel('horizontal_x')
plt.ylabel('Intensity')

plt.legend()

A physically correct $\alpha$ value will remove the phase fringes and boost the signal to noise of the reconstruction without losing real features in the sample, it's therefore important to carefully tune $\alpha$ for the specific sample you're applying it to.

#### References

[1] D. Paganin et al. "Simultaneous phase and amplitude extraction from a single defocused image of a homogeneous object." Journal of Microscopy, 206 (2002): 33-40.  [DOI:10.1046/j.1365-2818.2002.01010.x](https://doi.org/10.1046/j.1365-2818.2002.01010.x)

[2] F. De Carlo et al. “TomoBank: a tomographic data repository for computational x-ray science.” Measurement Science and Technology 29.3 (2018): 034004. [DOI:10.1088/1361-6501/aa9c19](http://www.doi.org/10.1088/1361-6501/aa9c19)

[3] https://tomobank.readthedocs.io/en/latest/source/data/docs.data.phasecontrast.html#wet-sample

[4] F. Brun et al. "A synchrotron radiation microtomography study of wettability and swelling of nanocomposite Alginate/Hydroxyapatite scaffolds for bone tissue engineering"  World Congress on Medical Physics and Biomedical Engineering, June 7-12, 2015, Toronto, Canada (pp.288-291) [DOI:10.1007/978-3-319-19387-8_70 ](http://dx.doi.org/10.1007/978-3-319-19387-8_70)

[5] Lawrence Berkeley National Laboratory, Centre for X-Ray Optics - X-Ray Interactions With Matter: Refractive Indices https://henke.lbl.gov/optical_constants/getdb2.html